# Programación declarativa @ URJC
# Programación funcional
## Curso 22-23, convocatoria extraordinaria (29 de junio de 2023)


# Definiciones auxiliares

In [59]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [2]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._


defined function foldTree

In [3]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard y Tipos Algebraicos de Datos

In [4]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A
}

defined trait Isomorphic

In [5]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1 (a)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica intuicionista proposicional: 

$\vdash p \wedge q \rightarrow \neg(\neg p \vee \neg q) $

In [6]:
def proof[P, Q]: (P, Q) => Not[Either[Not[P], Not[Q]]] = 
    (p: P, q: Q) => {
        case Left(np: Not[P]) => np(p)
        case Right(nq: Not[Q]) => nq(q)
    }

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c  \neg(\neg p \vee \neg q) \rightarrow p \wedge q $. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a las proposiciones $p$ y $q$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ p \vee \neg p, q \vee \neg q \} \vdash \neg(\neg p \vee \neg q) \rightarrow p \wedge q$

In [8]:
def proof[P, Q](lemp: Either[P, Not[P]], lemq: Either[Q, Not[Q]]): Not[Either[Not[P], Not[Q]]] => (P, Q) = 
    n => (lemp, lemq) match {
        case (Left(p), Left(q)) => (p, q)
        case (_, Right(nq)) => n(Right(nq))
        case (Right(np), _) => n(Left(np))
    }

defined function proof

# Ejercicio 1 (b)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica intuicionista proposicional: 

$\vdash p \wedge q \rightarrow \neg(p \rightarrow \neg q) $

In [8]:
def proof[P, Q]: (P, Q) => Not[P => Not[Q]] = 
    (p: P, q: Q) => (f: P => Not[Q]) => 
        f(p : P)(q : Q)

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c  \neg(p \rightarrow \neg q) \rightarrow p \wedge q $. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a las proposiciones $p$ y $q$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ p \vee \neg p, q \vee \neg q \} \vdash \neg(p \rightarrow \neg q)  \rightarrow p \wedge q$

In [9]:
def proof[P, Q](lemp: Either[P, Not[P]], lemq: Either[Q, Not[Q]]): Not[P => Not[Q]] => (P, Q) = 
    (n: (P => Q => Nothing) => Nothing) => (lemp, lemq) match {
        case (Left(p), Left(q)) => (p, q)
        case (_, Right(nq)) => n(_ => nq)
        case (Right(np), Left(q)) => (n((p: P) => (_ : Q) => np(p)), q)
    }

defined function proof

# Ejercicio 2 (a)
__(1,5 puntos)__

Demostrar que los tipos `(Either[P, Q], Either[R, S])` y `Either[(P, R), Either[(P, S), Either[(Q, R), (Q, S)]]]` son isomorfos, para todo tipo `P`, `Q`, `R` y `S`.

In [9]:
class Iso[P, Q, R, S] extends Isomorphic[(Either[P, Q], Either[R, S]), Either[(P, R), Either[(P, S), Either[(Q, R), (Q, S)]]]]{
    
    def from(t: (Either[P, Q], Either[R, S])): Either[(P, R), Either[(P, S), Either[(Q, R), (Q, S)]]] = 
        t match {
            case (Left(p), Left(r)) =>
                Left((p,r))
            case (Left(p), Right(s)) =>
                Right(Left((p,s)))
            case (Right(q), Left(r)) =>
                Right(Right(Left((q,r))))
            case (Right(q), Right(s)) =>
                Right(Right(Right((q,s))))
        }
    
    def to(e: Either[(P, R), Either[(P, S), Either[(Q, R), (Q, S)]]]): (Either[P, Q], Either[R, S]) = 
        e match {
            case Left((p, r)) =>
                (Left(p), Left(r))
            case Right(Left((p, s))) =>
                (Left(p), Right(s))
            case Right(Right(Left((q, r)))) =>
                (Right(q), Left(r))
            case Right(Right(Right((q, s)))) =>
                (Right(q), Right(s))
        }
}

defined class Iso

# Ejercicio 2 (b)
__(1,5 puntos)__

Demostrar que los tipos `((Y, Either[Z, V])) => X` y `(Z => Y => X, V => Y => X)` son isomorfos, para todo tipo `X`, `Y`, `Z` y `V`.

In [10]:
class Iso[X, Y, Z, V] extends Isomorphic[((Y, Either[Z, V])) => X, (Z => Y => X, V => Y => X)]{
    
    def from(t: ((Y, Either[Z, V])) => X): (Z => Y => X, V => Y => X) = 
        (z => y => t((y, Left(z))), 
         v => y => t((y, Right(v))))
    
    def to(t: (Z => Y => X, V => Y => X)): ((Y, Either[Z, V])) => X = {
        case (y, Left(z)) => t._1(z)(y)
        case (y, Right(v)) => t._2(v)(y)
    }
        
}

defined class Iso

# Ejercicio 3 (a)
__(5 puntos)__

__a) (1 punto)__ La función `duplicate` recibe un árbol de elementos cualesquiera y devuelve un árbol con la misma estructura donde los elementos de los nodos se encuentran duplicados. Por ejemplo: 

In [11]:
class TestDuplicate(duplicate: Tree[Int] => Tree[(Int, Int)]) extends FlatSpec with Matchers{
    
    "duplicate" should "work" in {
        duplicate(void) shouldBe void
        duplicate(node(left(leaf(1),3),4,left(right(3, leaf(9)),10))) shouldBe node(left(leaf((1,1)),(3,3)),(4,4),left(right((3,3), leaf((9,9))),(10,10)))
    }
}

defined class TestDuplicate

Implementa recursivamente la función `duplicate`.

In [12]:
def duplicate[A](t: Tree[A]): Tree[(A, A)] = 
    t match {
        case Empty() => Empty()
        case Node(left, root, right) => 
            Node(duplicate(left), (root, root), duplicate(right))
    }

defined function duplicate

In [13]:
run(new TestDuplicate(duplicate[Int]))

cmd10$Helper$TestDuplicate:
duplicate
- should work


__b) (1 punto)__ Implementa recursivamente la función `fusion` especificada de la siguiente forma: 

In [14]:
class TestFusion(fusion: Tree[(Int, Int, Int)] => Tree[Int]) extends FlatSpec with Matchers{
    
    "fusion" should "work" in {
        fusion(void) shouldBe void
        fusion(node(left(leaf((1,2,3)),(4,5,6)),(0,0,0),left(right((3,3,3), leaf((9,8,1))),(10,1,0)))) shouldBe 
               node(left(leaf(6)      ,15)     ,0      ,left(right(9      , leaf(18))     ,11))
    }
}

defined class TestFusion

Como se puede observar, la función `fusion` recibe un árbol cuyos elementos son tripletas de enteros, y devuelve un árbol con la misma estructura cuyos elementos resultan de sumar dichas tripletas.

In [15]:
def fusion(t: Tree[(Int, Int, Int)]): Tree[Int] = 
    t match {
        case Empty() => Empty()
        case Node(left, (a,b,c), right) => 
            Node(fusion(left), a+b+c, fusion(right))
    }

defined function fusion

In [16]:
run(new TestFusion(fusion))

cmd13$Helper$TestFusion:
fusion
- should work


__c) (1 punto)__ Se desean generalizar las dos funciones anteriores por medio de una función `map`. Esta función recibe un árbol de elementos cualquiera y devuelve un árbol con la misma estructura cuyos elementos resultan de aplicar una función recibida como parámetro a los elementos del árbol original. Por ejemplo: 

In [17]:
class TestMap(map: Tree[Int] => (Int => String) => Tree[String]) extends FlatSpec with Matchers{
    
    "map" should "work for _.toString" in {
        map(void)(_.toString) shouldBe void
        map(left(node(leaf(1),3,leaf(4)),4))(_.toString) shouldBe left(node(leaf("1"),"3",leaf("4")),"4")
    }
    
    "map" should "work for replicate('a')" in {
        
        def replicate(c: Char)(i: Int): String = 
            (1 to i).foldLeft("")((out: String, _) => out + c)
        
        map(void)(replicate('a')) shouldBe void
        map(left(node(leaf(1),3,leaf(4)),4))(replicate('a')) shouldBe left(node(leaf("a"),"aaa",leaf("aaaa")),"aaaa")
    }
}

defined class TestMap

In [18]:
def map[A, B](t: Tree[A])(f: A => B): Tree[B] = 
    t match {
        case Empty() => Empty()
        case Node(left, root, right) => 
            Node(map(left)(f), f(root), map(right)(f))
    }

defined function map

In [19]:
run(new TestMap(map))

cmd16$Helper$TestMap:
map
- should work for _.toString
map
- should work for replicate('a')


__d) (1 punto)__ Reimplementa las funciones de los apartados `a)` y `b)` utilizando la función `map`.

In [20]:
def duplicate[A](t: Tree[A]): Tree[(A, A)] = 
    map(t)(a => (a, a))

defined function duplicate

In [21]:
def fusion(t: Tree[(Int, Int, Int)]): Tree[Int] = 
    map(t){ case (a,b,c) => a+b+c }

defined function fusion

In [22]:
run(new TestDuplicate(duplicate[Int]))
run(new TestFusion(fusion))

cmd10$Helper$TestDuplicate:
duplicate
- should work
cmd13$Helper$TestFusion:
fusion
- should work


__e) (1 punto)__ Reimplementa la función `map` del apartado `c)` utilizando la función `foldTree`.

In [23]:
def map[A, B](t: Tree[A])(f: A => B): Tree[B] = 
    foldTree[A, Tree[B]](t)(
        Empty())((leftSol, root, rightSol) => Node(leftSol, f(root), rightSol))

defined function map

In [24]:
run(new TestMap(map))

cmd16$Helper$TestMap:
map
- should work for _.toString
map
- should work for replicate('a')


# Ejercicio 3 (b)
__(5 puntos)__

__a) (1 punto)__ La función `positives` recibe un árbol de elementos enteros y devuelve un árbol con la misma estructura donde los elementos de los nodos son valores booleanos que indican si el nodo correspondiente del árbol de entrada es positivo o no. Por ejemplo: 

In [34]:
class TestPositives(positives: Tree[Int] => Tree[Boolean]) extends FlatSpec with Matchers{
    
    "positives" should "work" in {
        positives(void) shouldBe void
        positives(node(left(leaf(-1),   3),   0,    left(right(-3,    leaf(-9)),   10))) shouldBe 
                  node(left(leaf(false),true),false,left(right(false, leaf(false)),true))
    }
}

defined class TestPositives

Implementa recursivamente la función `positives`.

In [35]:
def positives(t: Tree[Int]): Tree[Boolean] = 
    t match {
        case Empty() => Empty()
        case Node(left, root, right) => 
            Node(positives(left), root > 0, positives(right))
    }

defined function positives

In [37]:
run(new TestPositives(positives))

cmd33$Helper$TestPositives:
positives
- should work


__b) (1 punto)__ Implementa recursivamente la función `chars` especificada de la siguiente forma: 

In [38]:
class TestChars(chars: Tree[String] => Tree[List[Char]]) extends FlatSpec with Matchers{
    
    "chars" should "work" in {
        chars(void) shouldBe void
        chars(node(left(leaf("hola"),                "adios"),                   
                   "",     
                   left(right("pfff",                leaf("A")),      "bye"))) shouldBe 
              node(left(leaf(List('h','o','l','a')), List('a','d','i','o','s')), 
                   List(), 
                   left(right(List('p','f','f','f'), leaf(List('A'))), List('b','y','e')))
    }
}

defined class TestChars

Como se puede observar, la función `chars` recibe un árbol cuyos elementos son cadenas de caracteres, y devuelve un árbol con la misma estructura cuyos elementos se obtienen transformando el string correspondiente en una lista de caracteres. 

In [39]:
def chars(t: Tree[String]): Tree[List[Char]] = 
    t match {
        case Empty() => Empty()
        case Node(left, s, right) => 
            Node(chars(left), s.toList, chars(right))
    }

defined function chars

In [40]:
run(new TestFusion(fusion))

cmd13$Helper$TestFusion:
fusion
- should work


__c) (1 punto)__ Se desean generalizar las dos funciones anteriores por medio de una función `map`. Esta función recibe un árbol de elementos cualquiera y devuelve un árbol con la misma estructura cuyos elementos resultan de aplicar una función recibida como parámetro a los elementos del árbol original. Por ejemplo: 

In [41]:
class TestMap(map: Tree[Int] => (Int => String) => Tree[String]) extends FlatSpec with Matchers{
    
    "map" should "work for _.toString" in {
        map(void)(_.toString) shouldBe void
        map(left(node(leaf(1),3,leaf(4)),4))(_.toString) shouldBe left(node(leaf("1"),"3",leaf("4")),"4")
    }
    
    "map" should "work for replicate('a')" in {
        
        def replicate(c: Char)(i: Int): String = 
            (1 to i).foldLeft("")((out: String, _) => out + c)
        
        map(void)(replicate('a')) shouldBe void
        map(left(node(leaf(1),3,leaf(4)),4))(replicate('a')) shouldBe left(node(leaf("a"),"aaa",leaf("aaaa")),"aaaa")
    }
}

defined class TestMap

In [42]:
def map[A, B](t: Tree[A])(f: A => B): Tree[B] = 
    t match {
        case Empty() => Empty()
        case Node(left, root, right) => 
            Node(map(left)(f), f(root), map(right)(f))
    }

defined function map

In [43]:
run(new TestMap(map))

cmd40$Helper$TestMap:
map
- should work for _.toString
map
- should work for replicate('a')


__d) (1 punto)__ Reimplementa las funciones de los apartados `a)` y `b)` utilizando la función `map`.

In [44]:
def positives(t: Tree[Int]): Tree[Boolean] = 
    map(t)(_ > 0)

defined function positives

In [45]:
def chars(t: Tree[String]): Tree[List[Char]] = 
    map(t)(_.toList)

defined function chars

In [46]:
run(new TestPositives(positives))
run(new TestChars(chars))

cmd33$Helper$TestPositives:
positives
- should work
cmd37$Helper$TestChars:
chars
- should work


__e) (1 punto)__ Reimplementa la función `map` del apartado `c)` utilizando la función `foldTree`.

In [47]:
def map[A, B](t: Tree[A])(f: A => B): Tree[B] = 
    foldTree[A, Tree[B]](t)(
        Empty())((leftSol, root, rightSol) => Node(leftSol, f(root), rightSol))

defined function map

In [48]:
run(new TestMap(map))

cmd40$Helper$TestMap:
map
- should work for _.toString
map
- should work for replicate('a')


# Ejercicio 4 (a)
__(1,5 puntos)__

Considérese la siguiente implementación imperativa de la función `sumDivisors`, la cual calcula la suma de todos los divisores propios de un número dado (es decir, excluyendo a él mismo):

In [51]:
def sumDivisorsF(n: Int): Int = {
    var out = 0
    for (i <- 1 to n/2)
        if (n % i == 0) out += i
    out
}

defined function sumDivisorsF

A continuación se muestran algunos casos de prueba a modo de ejemplo de funcionamiento de la función:

In [52]:
class TestDivisors(sumDiv: Int => Int) extends FlatSpec with Matchers{
    
    "sum of divisors" should "work" in {
        sumDiv(2) shouldBe 1
        sumDiv(3) shouldBe 1
        sumDiv(4) shouldBe 3
        sumDiv(5) shouldBe 1
        sumDiv(6) shouldBe 6
        sumDiv(8) shouldBe 7
        sumDiv(9) shouldBe 4
    }
}

defined class TestDivisors

Implementa la función `sumDivisors` mediante la función de orden superior `foldLeft`. Para ello, téngase en cuenta que un rango de números `1 to n` puede transformarse en una lista mediante el método `toList`. Por ejemplo, `(1 to 3).toList == List(1,2,3)`.

In [53]:
def sumDivisors(n: Int): Int = 
    (1 to n/2).toList.foldLeft(0)( 
        (out, i) => if (n % i == 0) out + i else out
    )

// Realmente, el tipo `Range` de Scala también implementa el método `foldLeft`, por lo que se podría escribir simplemente: 
// (1 to n).foldLeft(...)(...)
// Lo mismo vale para otras funciones de orden superior, como `map`, `filter`, `exists`, etc.

defined function sumDivisors

In [54]:
run(new TestDivisors(sumDivisors))

cmd51$Helper$TestDivisors:
sum of divisors
- should work


# Ejercicio 4 (b)
__(1,5 puntos)__

Considérese la siguiente implementación imperativa de la función `sumDivisors`, la cual calcula la suma de todos los divisores propios de un número dado (es decir, excluyendo a él mismo):

In [51]:
def sumDivisorsF(n: Int): Int = {
    var out = 0
    for (i <- 1 to n/2)
        if (n % i == 0) out += i
    out
}

defined function sumDivisorsF

A continuación se muestran algunos casos de prueba a modo de ejemplo de funcionamiento de la función:

In [52]:
class TestDivisors(sumDiv: Int => Int) extends FlatSpec with Matchers{
    
    "sum of divisors" should "work" in {
        sumDiv(2) shouldBe 1
        sumDiv(3) shouldBe 1
        sumDiv(4) shouldBe 3
        sumDiv(5) shouldBe 1
        sumDiv(6) shouldBe 6
        sumDiv(8) shouldBe 7
        sumDiv(9) shouldBe 4
    }
}

defined class TestDivisors

Implementa la función `sumDivisors` utilizando la función de order superior `filter`.

In [57]:
def sumDivisors(n: Int): Int = 
    (1 to n/2).toList.filter(n % _ == 0).foldLeft(0)(_ + _)

// Una implementación más idiomática en Scala sería: (1 to n/2).filter(n%_==0).reduce(_+_)

defined function sumDivisors

In [58]:
run(new TestDivisors(sumDivisors))

cmd51$Helper$TestDivisors:
sum of divisors
- should work
